In [1]:
from sqlalchemy import create_engine
import urllib

#step1: connect to postgresql

username = "postgres"
raw_password = "#@giZHini0301"
host = "localhost"
port = "5432"
database = "sql_projects"
password = urllib.parse.quote_plus(raw_password) 
#print(repr(host))


#engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")
engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)


#step 2 load dataframe into postgresql

#table_name = "netflix_shows"
#df.to_sql(table_name, engine, if_exists="replace", index= False)

table_name = "superstore"

In [2]:
import pandas as pd

pd.read_sql("SELECT 1;", engine)

,?column?
0,1


In [3]:
df = pd.read_sql("select * from superstore",engine)

In [5]:
df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,order_date_raw,ship_date_raw
0,1,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,09-11-2013,12-11-2013
1,2,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,09-11-2013,12-11-2013
2,3,CA-2013-138688,2013-06-13,2013-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,13-06-2013,17-06-2013
3,4,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,11-10-2012,18-10-2012
4,5,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,11-10-2012,18-10-2012


In [6]:
df.shape

(9994, 23)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          9994 non-null   int64         
 1   order_id        9994 non-null   object        
 2   order_date      9994 non-null   datetime64[ns]
 3   ship_date       9994 non-null   datetime64[ns]
 4   ship_mode       9994 non-null   object        
 5   customer_id     9994 non-null   object        
 6   customer_name   9994 non-null   object        
 7   segment         9994 non-null   object        
 8   country         9994 non-null   object        
 9   city            9994 non-null   object        
 10  state           9994 non-null   object        
 11  postal_code     9994 non-null   int64         
 12  region          9994 non-null   object        
 13  product_id      9994 non-null   object        
 14  category        9994 non-null   object        
 15  sub_

In [8]:
df.isnull().sum()

row_id            0
order_id          0
order_date        0
ship_date         0
ship_mode         0
customer_id       0
customer_name     0
segment           0
country           0
city              0
state             0
postal_code       0
region            0
product_id        0
category          0
sub_category      0
product_name      0
sales             0
quantity          0
discount          0
profit            0
order_date_raw    0
ship_date_raw     0
dtype: int64

select Category, sum(sales) sales_amount 
	from superstore
	group by Category
	order by 2 desc
    

In [17]:
result1 = df.groupby('category', as_index=False)['sales'].sum().rename(columns={'sales':'sales_amount'}).sort_values(by='sales_amount', ascending=False)
result1

,category,sales_amount
2,Technology,836154.0330
0,Furniture,741999.7953
1,Office Supplies,719047.0320


select customer_id, customer_name, sum(profit) total_profit
	from superstore
	group by customer_id, customer_name
	order by 2 desc limit 5

In [22]:
result2= (
    df.groupby(['customer_id', 'customer_name'], as_index=False)['profit'].sum()
.rename(columns = {'profit':'total_profit'}).sort_values(by='total_profit', ascending = False).head(5)
)

result2

,customer_id,customer_name,total_profit
741,TC-20980,Tamara Chand,8981.3239
621,RB-19360,Raymond Buch,6976.0959
669,SC-20095,Sanjit Chand,5757.4119
327,HL-15040,Hunter Lopez,5622.4292
6,AB-10105,Adrian Barton,5444.8055


select DATE_TRUNC('month', order_date) as month, round(sum(sales)::numeric,2) total_sales
	from superstore
	group by month
	order by 1 

In [ ]:
df['month'] = df['order_date'].dt.to_period('M').dt.to_timestamp()

result3 = (
    df.groupby('month', as_index=False)['sales'].sum()
    .rename(columns={'sales':'total_sales'})
        .assign(total_sales=lambda x:x['total_sales'].round(2))
    .sort_values('month', ascending=False)
    
)

result3

select sub_category, round(avg(discount)::numeric, 2) average
	from  superstore
	group by 1
	order by 2

In [ ]:
result4 = (
    df.groupby('sub_category',as_index=False)['discount'].mean()
    .rename(columns={'discount':'average_discount'})
    .assign(average_discount = lambda x:x['average_discount'].round(2))
    .sort_values('average_discount', ascending=False)
    )
result4

select sub_category,
	round(sum(profit) ::numeric,2) total_profit,
	round(
	(100 * sum(profit) / sum(sum(profit))over())::numeric,2) total_profit_percentage
from superstore
group by sub_category
order by 3 desc
	

In [ ]:
result5 = df.groupby('sub_category', as_index=False).agg(total_profit=('profit','sum')).round(2)
result5

In [ ]:
result5['total_profit_percent'] = (
    100* result5['total_profit'] / result5['total_profit'].sum()
).round(2)

result5

In [ ]:
result5 = result5.sort_values('total_profit_percent', ascending=False)
result5

select customer_id, customer_name,
	max(order_date) recent_purchase,
	count(distinct order_id) frequency_order,
	round(sum(sales)::numeric,2) monetary
from superstore
group by 1, 2
order by 1


In [ ]:
result6 = (
    df.groupby(['customer_id','customer_name'], as_index=False)
    .agg(monetary = ('sales','sum'), recent_purchase = ('order_date','max'), frequency_order = ('order_id','nunique'))
    .sort_values('frequency_order', ascending=False)
    )
result6


select product_id, product_name, 
	round(sum(sales)::numeric,2) total_sales, round(sum(profit)::numeric,2) total_profit
from superstore
group by 1,2 
having sum(profit)>0
order by 4 desc


In [ ]:
result7 = (
    df.groupby(['product_id','product_name'], as_index=False)
    .agg(total_sales = ('sales','sum'),total_profit = ('profit','sum')).round(2)
    .query('total_profit > 0')
    .sort_values('total_profit', ascending=False)
)
result7

with cte as (
	select product_name, region, sum(profit) total_profit,
	row_number() over(partition by region order by sum(profit) desc ) ranking
	
	from superstore
	--where region <>'Central'
	group by product_name, region
	order by 2 
)

select * from cte
where ranking = 1

In [61]:
cte = (
    df.groupby(['product_name','region'], as_index=False)
    .agg(total_profit= ('profit','sum'))
)

cte['ranking'] = (
    cte.sort_values([ 'total_profit'],ascending=False)
    .groupby('region')
    .cumcount()+1
)

cte

,product_name,region,total_profit,ranking
0,"""While you Were Out"" Message Book, One Form pe...",Central,3.3390,779
1,"""While you Were Out"" Message Book, One Form pe...",South,3.3390,782
2,"""While you Were Out"" Message Book, One Form pe...",West,3.7100,1253
3,"#10 Gummed Flap White Envelopes, 100/Box",Central,5.3690,712
4,"#10 Gummed Flap White Envelopes, 100/Box",East,11.3988,798
...,...,...,...,...
5216,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,East,90.6696,253
5217,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,West,55.4092,428
5218,netTALK DUO VoIP Telephone Service,Central,62.9880,227
5219,netTALK DUO VoIP Telephone Service,East,52.4900,367


In [62]:
result8 = cte[cte['ranking']==1]
result8

,product_name,region,total_profit,ranking
1155,Canon imageCLASS 2200 Advanced Copier,Central,8399.9760,1
1156,Canon imageCLASS 2200 Advanced Copier,East,10079.9712,1
1157,Canon imageCLASS 2200 Advanced Copier,West,6719.9808,1
1841,Fellowes PB500 Electric Punch Plastic Comb Bin...,South,3812.9700,1
